In [ ]:
pip install opencv-python numpy playsound

In [1]:
import cv2
import numpy as np
import playsound
import time

SAFE_COFFEE_LIMIT = 300  # mg
AVG_GULP_VOLUME = 50  # ml 
CAFFEINE_PER_ML = 0.012  # mg/mL 
current_intake = 0
num_gulps = 0
num_cups = 0
MAX_GULPS_PER_CUP = 10  # I generally take 10 gulps to finish my cup
last_gulp_time = 0 
GULP_THRESHOLD_TIME = 2  # Minimum seconds between gulps ( I take my coffee slow xD )

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

def main():
    global current_intake, num_gulps, num_cups, last_gulp_time  
    cap = cv2.VideoCapture(0)  

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

        lower_green = np.array([35, 80, 120])
        upper_green = np.array([85, 255, 255]) #my coffee mug is green, so wrote this. Put in your coffee mug's color

        mask = cv2.inRange(hsv, lower_green, upper_green)
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        mug_near_mouth = False 

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

        for (x, y, w, h) in faces:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
            roi_gray = gray[y:y + h, x:x + w]
            roi_color = frame[y:y + h, x:x + w]
            mouth_y = y + int(0.7 * h)
            mouth_region = (x, mouth_y, w, int(0.3 * h))
            cv2.rectangle(frame, (mouth_region[0], mouth_region[1]), 
                          (mouth_region[0] + mouth_region[2], mouth_region[1] + mouth_region[3]), 
                          (0, 255, 0), 2)

            if contours:
                mug_contour = max(contours, key=cv2.contourArea)
                mug_x, mug_y, mug_w, mug_h = cv2.boundingRect(mug_contour)
                cv2.drawContours(frame, [mug_contour], -1, (0, 255, 0), 2)

                if (mouth_region[1] < mug_y < mouth_region[1] + mouth_region[3] and
                        mouth_region[0] < mug_x + mug_w//2 < mouth_region[0] + mouth_region[2]):
                    mug_near_mouth = True
                    if time.time() - last_gulp_time > GULP_THRESHOLD_TIME:
                        num_gulps += 1
                        current_intake += AVG_GULP_VOLUME * CAFFEINE_PER_ML
                        last_gulp_time = time.time()
                        if num_gulps >= MAX_GULPS_PER_CUP:
                            num_cups += 1
                            num_gulps = 0
                        if current_intake >= SAFE_COFFEE_LIMIT:
                            print("Maximum coffee intake reached!")
                            playsound.playsound('./alert_sound.mp3')  # Added a sound to alert me. Replace with your sound or completely remove this part
                            break

        # Display statistics on screen
        cv2.putText(frame, f"Intake: {current_intake:.2f} mg", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
        cv2.putText(frame, f"Gulps: {num_gulps}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
        cv2.putText(frame, f"Cups: {num_cups}", (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

        # Show the original frame and the mask
        cv2.imshow('Coffee Limiter', frame)
        cv2.imshow('Mask', mask)

        # Exit condition
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()
